In [3]:
import pandas as pd

chemin_fichier = 'parkinsons_updrs.data'

data = pd.read_csv(chemin_fichier, sep=',', header=0)

# Affichez les premières lignes pour vérifier si les données ont été chargées correctement
data.columns

# drop des colonnes nulles
data = data.dropna(axis=0)
data = data.drop(data.columns[0], axis=1)
data.head()


,age,sex,test_time,motor_UPDRS,total_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,Jitter:DDP,...,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
0,72,0,5.6431,28.199,34.398,0.00662,0.000034,0.00401,0.00317,0.01204,...,0.230,0.01438,0.01309,0.01662,0.04314,0.014290,21.640,0.41888,0.54842,0.16006
1,72,0,12.6660,28.447,34.894,0.00300,0.000017,0.00132,0.00150,0.00395,...,0.179,0.00994,0.01072,0.01689,0.02982,0.011112,27.183,0.43493,0.56477,0.10810
2,72,0,19.6810,28.695,35.389,0.00481,0.000025,0.00205,0.00208,0.00616,...,0.181,0.00734,0.00844,0.01458,0.02202,0.020220,23.047,0.46222,0.54405,0.21014
3,72,0,25.6470,28.905,35.810,0.00528,0.000027,0.00191,0.00264,0.00573,...,0.327,0.01106,0.01265,0.01963,0.03317,0.027837,24.445,0.48730,0.57794,0.33277
4,72,0,33.6420,29.187,36.375,0.00335,0.000020,0.00093,0.00130,0.00278,...,0.176,0.00679,0.00929,0.01819,0.02036,0.011625,26.126,0.47188,0.56122,0.19361


function to calculate all deferent metrics
r square , mean_square error and absolute error and correlation

In [6]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

def eval(y_true,y_pred):
  r2 = r2_score(y_true, y_pred)
  print('the r square is  : ',r2)
  rmse = np.sqrt(mean_squared_error(y_true, y_pred))
  print("the rmse value is:",rmse)
  mae = mean_absolute_error(y_true, y_pred)
  print('the mae mean absolute error',mae)
  corr = np.corrcoef(y_true, y_pred)[0, 1]
  print("the coloration value is : ",corr)


USING MODEL KNeighborsRegressor


In [27]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor



# Extraction des caractéristiques et des étiquettes
X = data.drop(columns=['motor_UPDRS', 'total_UPDRS'])  # Caractéristiques

y_updrs = data[['motor_UPDRS','total_UPDRS']]  # Étiquette 1



# Normalisation des caractéristiques
scaler_X = MinMaxScaler()
X_normalized = scaler_X.fit_transform(X)

# Normalisation des étiquettes 
scaler_y = MinMaxScaler()
y_normalized_UPDRS = scaler_y.fit_transform(y_updrs)

# Division des données en ensembles d'entraînement et de test
x_train, x_test, y_train, y_test = train_test_split(X_normalized, y_normalized_UPDRS, test_size=0.2, random_state=0)

# Création et entraînement du modèle KNeighborsRegressor
for i in range(8,20):
  model = KNeighborsRegressor(n_neighbors=i)
  model.fit(x_train, y_train)

  # Évaluation du modèle
  score = model.score(x_train, y_train)
  scoretest  = model.score(x_test,y_test)
  print(f"Score using {i} neighbors R² du modèle train  : {score}")
  print(f"Score {i}  neighbots R² du modèle test : {scoretest}")
  print("------------")


Score using 8 neighbors R² du modèle train  : 0.7560489322313189
Score 8  neighbots R² du modèle test : 0.7079668566323676
------------
Score using 9 neighbors R² du modèle train  : 0.7464858283544002
Score 9  neighbots R² du modèle test : 0.7037157548728392
------------
Score using 10 neighbors R² du modèle train  : 0.7370837724693671
Score 10  neighbots R² du modèle test : 0.6929152932272254
------------
Score using 11 neighbors R² du modèle train  : 0.7276817644833952
Score 11  neighbots R² du modèle test : 0.6763162335324873
------------
Score using 12 neighbors R² du modèle train  : 0.7213111071149496
Score 12  neighbots R² du modèle test : 0.6704035694488423
------------
Score using 13 neighbors R² du modèle train  : 0.7132876628042308
Score 13  neighbots R² du modèle test : 0.6698778905483788
------------
Score using 14 neighbors R² du modèle train  : 0.7039870931358139
Score 14  neighbots R² du modèle test : 0.6635362814405978
------------
Score using 15 neighbors R² du modèle 

SEARCHING FOR THE BEST PARAMETRE,
WE USE GridSearchcv

In [7]:
from sklearn.model_selection import train_test_split, GridSearchCV

# Définir la grille des paramètres à tester
param_grid = {
    'n_neighbors': [i for i in range(6,18)],  # Valeurs pour le nombre de voisins
    'metric': ['euclidean', 'manhattan'],  # Différentes métriques de distance
    'weights': ['uniform', 'distance']  # Différentes options pour les poids
}
grid_search = GridSearchCV(model, param_grid, cv=5)
# Entraînement du modèle avec recherche des meilleurs paramètres
grid_search.fit(x_train, y_train)

# Afficher les détails des scores pour chaque combinaison de paramètres
results = grid_search.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Score moyen : {mean_score} - Paramètres : {params}")



Score moyen : 0.6672331751690772 - Paramètres : {'metric': 'euclidean', 'n_neighbors': 6, 'weights': 'uniform'}
Score moyen : 0.6825298628510333 - Paramètres : {'metric': 'euclidean', 'n_neighbors': 6, 'weights': 'distance'}
Score moyen : 0.6620401467498578 - Paramètres : {'metric': 'euclidean', 'n_neighbors': 7, 'weights': 'uniform'}
Score moyen : 0.6789222187526376 - Paramètres : {'metric': 'euclidean', 'n_neighbors': 7, 'weights': 'distance'}
Score moyen : 0.6609713613674761 - Paramètres : {'metric': 'euclidean', 'n_neighbors': 8, 'weights': 'uniform'}
Score moyen : 0.6781422132735822 - Paramètres : {'metric': 'euclidean', 'n_neighbors': 8, 'weights': 'distance'}
Score moyen : 0.6562179548048079 - Paramètres : {'metric': 'euclidean', 'n_neighbors': 9, 'weights': 'uniform'}
Score moyen : 0.6744991259830242 - Paramètres : {'metric': 'euclidean', 'n_neighbors': 9, 'weights': 'distance'}
Score moyen : 0.6434096184384811 - Paramètres : {'metric': 'euclidean', 'n_neighbors': 10, 'weights'

In [13]:
# Afficher les meilleurs paramètres trouvés
print("Meilleurs paramètres:", grid_search.best_params_)

# Évaluation du modèle avec les meilleurs paramètres sur les données de test
best_model = grid_search.best_estimator_
score = best_model.score(x_test, y_test)

y_pred=best_model.predict(x_test)

print(f"Score R² test du modèle avec les meilleurs paramètres : {score}")
eval(y_test,y_pred)

Meilleurs paramètres: {'metric': 'manhattan', 'n_neighbors': 6, 'weights': 'distance'}
Score R² test du modèle avec les meilleurs paramètres : 0.7644510515339236
the r square is  :  0.7644510515339236
the rmse value is: 0.11024377712372554
the mae mean absolute error 0.06535725350839787
the coloration value is :  -1.0


the best parametre is : {'metric': 'manhattan', 'n_neighbors': 6, 'weights': 'distance'}

take an example to predict

In [9]:
print(X_normalized.shape)
test = best_model.predict([X_normalized[1,:]])
test = scaler_y.inverse_transform(test)
print(test)

(5875, 19)
[[21.1021611  26.86497434]]


using svm algo

In [11]:
from sklearn.svm import SVR

In [25]:
from sklearn.multioutput import MultiOutputRegressor
# using the multioutputt the svm doesnt support multi regression
# Create an SVR model
svr_model = SVR(kernel='rbf',C=100)

# Wrap the SVR model with MultiOutputRegressor
multioutput_model = MultiOutputRegressor(svr_model)

# Train the multi-output model
multioutput_model.fit(x_train, y_train)

# Make predictions
y_pred = multioutput_model.predict(x_test)

# Evaluate the performance
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared",mse)
eval(y_test,y_pred)
test = multioutput_model.predict([X_normalized[1,:]])
test = scaler_y.inverse_transform(test)
print(test)

Mean Squared 0.021774184742448445
the r square is  :  0.5787559581663202
the rmse value is: 0.147560783213049
the mae mean absolute error 0.11300990486148446
the coloration value is :  -1.0
[[25.12787475 32.63338159]]


decission tree

In [28]:
from sklearn.tree import DecisionTreeRegressor

# Creating and training the Decision Tree Regression model
tree_regressor = DecisionTreeRegressor(max_depth=11)  # adjust the max_depth parameter
tree_regressor.fit(x_train, y_train)

# Making predictions on the test set
y_pred = tree_regressor.predict(x_test)

# Evaluating the model
mse = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r_squared)
eval(y_test,y_pred)
test = multioutput_model.predict([X_normalized[1,:]])
test = scaler_y.inverse_transform(test)
print(test)

Mean Squared Error: 0.0036869396963509172
R-squared: 0.928597064563663
the r square is  :  0.928597064563663
the rmse value is: 0.06072017536495524
the mae mean absolute error 0.024113214557162484
the coloration value is :  -1.0
[[25.12787475 32.63338159]]
